In [ ]:
%load_ext autoreload
%autoreload 2
from config import username, password
from modules.login_module import logIn, launch_to_homescreen, create_directory
from modules.download_files_module import request_report_process, download_loop_missing, download_process,  move_files_over, unzip_xlsx_file, unzip_files_in_same_dir, move_xlsx_files
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
import time
from datetime import datetime
from modules.sql_query_module import SQL_query
import urllib
import sqlalchemy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
today_date = datetime.now()
formatted_month_day = today_date.strftime("%m_%d")
pd.set_option('display.max_columns', None)

logging.basicConfig(filename='ELPAC_SBAC_results.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)
logging.info('\n\n-------------ELPAC_SBAC_results new instance log')

#create file_download dir, and establish download directory
create_directory('file_downloads')
create_directory(f'file_downloads\\sbac\\{formatted_month_day_year}')
create_directory(f'file_downloads\\elpac\\{formatted_month_day_year}')

# Set up Chrome options
download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day_year}'
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_directory,
         'profile.default_content_setting_values.automatic_downloads': 1,
         'profile.content_settings.exceptions.automatic_downloads.*.setting': 1}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

logIn(username, password, driver)
launch_to_homescreen(driver)

coord_list = ['LEA CAASPP Coordinator at Alain Leroy Locke College Preparatory Academy',
 'LEA CAASPP Coordinator at Animo City of Champions Charter High',
 'LEA CAASPP Coordinator at Animo Compton Charter',
 'LEA CAASPP Coordinator at Animo Ellen Ochoa Charter Middle',
 'LEA CAASPP Coordinator at Animo Florence-Firestone Charter Middle',
 'LEA CAASPP Coordinator at Animo Inglewood Charter High',
 'LEA CAASPP Coordinator at Animo Jackie Robinson High',
 'LEA CAASPP Coordinator at Animo James B. Taylor Charter Middle',
 'LEA CAASPP Coordinator at Animo Jefferson Charter Middle',
 'LEA CAASPP Coordinator at Animo Leadership High',
 'LEA CAASPP Coordinator at Animo Legacy Charter Middle',
 'LEA CAASPP Coordinator at Animo Mae Jemison Charter Middle',
 'LEA CAASPP Coordinator at Animo Pat Brown',
 'LEA CAASPP Coordinator at Animo Ralph Bunche Charter High',
 'LEA CAASPP Coordinator at Animo South Los Angeles Charter',
 'LEA CAASPP Coordinator at Animo Venice Charter High',
 'LEA CAASPP Coordinator at Animo Watts College Preparatory Academy',
 'LEA CAASPP Coordinator at Oscar De La Hoya Animo Charter High',
 'LEA ELPAC Coordinator at Alain Leroy Locke College Preparatory Academy',
 'LEA ELPAC Coordinator at Animo City of Champions Charter High',
 'LEA ELPAC Coordinator at Animo Compton Charter',
 'LEA ELPAC Coordinator at Animo Ellen Ochoa Charter Middle',
 'LEA ELPAC Coordinator at Animo Florence-Firestone Charter Middle',
 'LEA ELPAC Coordinator at Animo Inglewood Charter High',
 'LEA ELPAC Coordinator at Animo Jackie Robinson High',
 'LEA ELPAC Coordinator at Animo James B. Taylor Charter Middle',
 'LEA ELPAC Coordinator at Animo Jefferson Charter Middle',
 'LEA ELPAC Coordinator at Animo Leadership High',
 'LEA ELPAC Coordinator at Animo Legacy Charter Middle',
 'LEA ELPAC Coordinator at Animo Mae Jemison Charter Middle',
 'LEA ELPAC Coordinator at Animo Pat Brown',
 'LEA ELPAC Coordinator at Animo Ralph Bunche Charter High',
 'LEA ELPAC Coordinator at Animo South Los Angeles Charter',
 'LEA ELPAC Coordinator at Animo Venice Charter High',
 'LEA ELPAC Coordinator at Animo Watts College Preparatory Academy',
 'LEA ELPAC Coordinator at Oscar De La Hoya Animo Charter High']

elpac_coordinators = [coord for coord in coord_list if 'ELPAC' in coord]
caaspp_coordinators = [coord for coord in coord_list if 'CAASPP' in coord]

#This exists when passing names into the requested reports, as a subset. Change list into a set to only retain unique schools
school_report_names = [entry.split(' at ', 1)[1] for entry in elpac_coordinators]
school_report_names = list(set(school_report_names))

# ---------------------------------------SBAC Files Request and Download

# Call the function, school report names variable is called for just school name
#Equivalent of Student Score Data File
request_report_process(driver, 'SBAC', 'CAASPP_Student_Score_Data_Extract_Report', caaspp_coordinators, '2023')
download_process(school_report_names, '2023 CAASPP Student Score Data File By Enrolled LEA', driver) 

#This is here three times to see if anything got skipped the first time. Initial dir is set at ELPAC only to move the files over to SBAC dir
#Will run 5 times

time.sleep(10) #implemented to give time for files to download
download_loop_missing(f'elpac\\{formatted_month_day_year}', '2023 CAASPP Student Score Data File By Enrolled LEA', driver)

#This moves the files from ELPAC  timestamp dir to SBAC timestamp dir. 
#This is because the download dir cannot be changed in Selenium
move_files_over()

# --------------------------------------------ELPAC Files Request and Download

driver.switch_to.default_content()
request_report_process(driver, 'ELPAC', 'Student_Results_Report_Student_Score_Data_Extract', elpac_coordinators, '2023')
download_process(school_report_names, '2023 Summative ELPAC and Summative Alternate ELPAC Student Score Data File By Enrolled LEA', driver) 

time.sleep(10) #implemented to give time for files to download
#This is here three times to see if anything got skipped the first time. 
#Dir remains ELPAC for constant download directory
download_loop_missing(f'elpac\\{formatted_month_day_year}', '2023 Summative ELPAC and Summative Alternate ELPAC Student Score Data File By Enrolled LEA', driver)

#Close out driver window once done
driver.close()

#Takes 14 mins to run up to this point
# -----------------------------------------Unzip the Files and Move them to the P-Drive in this location 'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'
unzip_files_in_same_dir('elpac')
unzip_files_in_same_dir('sbac')

#Keeps raw zip files in the same dir. Only moves over xlsx files
try:
    move_xlsx_files('sbac')
    logging.info('Moved SBAC XLSX files to p-drive')
except:
    logging.info('Unable to move SBAC XLSX files to the p-drive, must be connected to the VPN')
try:
    move_xlsx_files('elpac')
    logging.info('Moved ELPAC XLSX files to p-drive')
except:
    logging.info('Unable to move ELPAC XLSX files to the p-drive, must be connected to the VPN')


#Checks these dirs, for all files being there
test_instance = TestFileProcessing()
test_instance.test_file_processing('sbac')
test_instance.test_file_processing('elpac')

# #Takes roughly 25 mins to download and send
# #Must be connected to the p-drive

# ---------------------------------STACKING & SENDING FILES----------------------------------

directory_path = r'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'
formatted_month_day_year = today_date.strftime("%m_%d_%y")


directory_path_sbac = os.path.join(directory_path, f'sbac_{formatted_month_day_year}')
sbac = stack_files(directory_path_sbac)
sbac['CALPADSSchoolCode'] = sbac['CALPADSSchoolCode'].astype(str).str[7:]


directory_path_elpac = os.path.join(directory_path, f'elpac_{formatted_month_day_year}')
elpac = stack_files(directory_path_elpac)
elpac['CALPADSSchoolCode'] = elpac['CALPADSSchoolCode'].astype(str).str[7:]

sbac = filter_on_full_cds_code(sbac, 'CALPADSSchoolCode')
elpac = filter_on_full_cds_code(elpac, 'CALPADSSchoolCode')


# -------------------------------------------get_elpac_import------------------
elpac = get_elpac_import(elpac)
elpac.name = 'ELPAC'
directory_path_elpac = os.path.join(directory_path, f'ELPAC_STACKED_{formatted_month_day_year}.csv')
try:
    elpac.to_csv(directory_path_elpac, index=False)
    logging.info(f'ELPAC sent to {directory_path} for ellevation pickup')
except:
    logging.info(f'ELPAC unable to send for ellevation pickup')


#Differing decoding method. Refer to message with Abi
# ss_decode = {4.0: 'WelDev', 
#                 3.0: 'ModDev', 
#                 2.0: 'SomDev',
#                 1.0: 'MinDev', 
#                 '': 'No Score', 
#                 'NS': 'No Score'}

# -------------------------------Get SBAC import-------------------------

#Missing PLScore Column and ProficiencyLevelCode Mapping
sbac_final = get_sbac_cols(sbac, 'SBAC')
sbac_final.name = 'SBAC'
directory_path_sbac = os.path.join(directory_path, f'SBAC_STACKED_{formatted_month_day_year}.csv')
try:
    sbac_final.to_csv(directory_path_sbac, index=False)
    logging.info(f'SBAC sent to {directory_path} for ellevation pickup')
except:
    logging.info(f'SBAC unable to send for ellevation pickup')

# PL Score 1	STNM
# PL Score 2	STNL
# PL Score 3	STMT
# PL Score 4	STEX

# ------------------------------Get CAST import-------------------------

cast = get_cast_cols(sbac, 'CAST')
cast.name = 'CAST'
directory_path_cast = os.path.join(directory_path, f'CAST_STACKED_{formatted_month_day_year}.csv')
try:
    cast.to_csv(directory_path_cast, index=False)
    logging.info(f'CAST sent to {directory_path} for ellevation pickup')
except:
    logging.info(f'CAST unable send for ellevation pickup')

# PL Score 1	BLST
# PL Score 2	ANST
# PL Score 3	ABST

# ------------------------------------------------------send to DataTeamSandbox on 89 server-----------------------------


In [23]:

cast = pd.read_csv(r'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\CAST_STACKED_04_26_24.csv')
elpac = pd.read_csv(r'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\ELPAC_STACKED_04_26_24.csv')
sbac_final = pd.read_csv(r'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\SBAC_STACKED_04_26_24.csv')

In [24]:
cast.head(1)

,Abbreviation,SchoolID,MasterSchoolID,StudentID,SSID,TestDate,TestType,TestPeriod,TestSubjectGroup,TestSubject,TestGrade,TestName,TestScoreType,AssessmentName,RawScore,ScaleScore
0,Animo Ellen Ochoa Charter Middle,NaN,NaN,NaN,1367208652,2024-04-30,CAST,NaN,Science,Science- Overall,8,CAST-Overall,NaN,Grade 8 CAST,386.0,386.0


In [25]:
elpac.head(1)

#assimilate the columns. Make all of them the same. Void prior process. 

,Abbreviation,SchoolID,MasterSchoolID,StudentNumber,StudentID,SSID,TestGrade,ELStatus,TestDate,DisplayDate,TestType,TestPeriod,TestScoreType,testname,ScaleScore,PLScore,ProficiencyLevelCode
0,Oscar De La Hoya Animo Charter High,NaN,NaN,61922,NaN,8584196218,11,Yes,2024-04-30,NaN,ELPAC,NaN,NaN,ELPAC-Overall,1566.0,3.0,WelDev


# Debug

In [18]:
%load_ext autoreload
%autoreload 2
from config import username, password
from modules.login_module import logIn, launch_to_homescreen, create_directory
from modules.download_files_module import request_report_process, download_loop_missing, download_process,  move_files_over, unzip_xlsx_file, unzip_files_in_same_dir, move_xlsx_files
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
import time
from datetime import datetime
from modules.sql_query_module import SQL_query
import urllib
import sqlalchemy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
today_date = datetime.now()
formatted_month_day = today_date.strftime("%m_%d")
pd.set_option('display.max_columns', None)

cast = pd.read_csv('cast.csv')
elpac = pd.read_csv('elpac.csv')
sbac_final = pd.read_csv('sbac.csv')

# instantiate class with elpac variable, clean up dtypes
def grab_and_send_new(file, file_name):

    file_obj = Clean(file, file_name)
    file_obj_clean = file_obj.clean_up_rotating_file()
    merging_cols = ['SSID', 'TestType', 'testname', 'ScaleScore']
    new_records = SQL_query.obtain_new(elpac_clean, file_name, merging_cols)

    dtypes, table_cols = SQL_query.get_dtypes(file, 'DataTeamSandbox', f'{file_name}_Scores')

    return(new_records)

    if new_records.empty == True:
        logging.info(f'No new records to insert, {file_name} file is empty')
    else:
        file.to_csv(f'file_downloads/{file_name}_new_records.csv')
        logging.info(f'{file_name}_new_records.csv sent to file downloads')
        try:
            new_records.to_sql(f'{file_name}_Scores', schema='dbo', con = SQL_query.engine, if_exists = 'append', index = False, dtype=dtypes)
            logging.info(f'Appending {len(new_records)} records to {file_name}_Scores')

        except Exception as e:
            logging.info(f'Unable to append to table {file_name}_Scores due to \n {e}')



#debug again to see why the dtypes are being changes
# grab_and_send_new(elpac, 'ELPAC')
grab_and_send_new(cast, 'CAST')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


KeyError: "['StudentNumber', 'ELStatus', 'testname', 'PLScore', 'ProficiencyLevelCode'] not in index"

In [20]:
# cast.head(1)
#Looks like the csv version of cast did not get saved properly 
"['StudentNumber', 'ELStatus', 'testname', 'PLScore', 'ProficiencyLevelCode'] not in index"

#clean up rotating file is going to have some specific column names depending on cast or sbac

,Abbreviation,SchoolID,MasterSchoolID,StudentID,SSID,TestDate,TestType,TestPeriod,TestSubjectGroup,TestSubject,TestGrade,TestName,TestScoreType,AssessmentName,RawScore,ScaleScore
0,Oscar De La Hoya Animo Charter High,NaN,NaN,NaN,2386978391,2024-03-31,CAST,NaN,Science,Science- Overall,11,CAST-Overall,NaN,Grade 11 CAST,633.0,633.0


In [ ]:
#['SSID', 'TestType', 'testname', 'ScaleScore'] are all present in SBAC Scores & CAST Scores

In [ ]:
#send this file to Abi. 
#original name mapping to new columns

#Abbreviation - map on CALPADSDIstrictName
#schoolid blank
#MasterSchoolID blank
#StudentNumber - LocalStudentId
#StudentID blank
#SSID - SSID
#TestGrade - GradeAssessed
#ELStatus Blank
#TestDate - End of the month for the month uploaded
#DisplayDate blank
#testtype - 21 (General ELPAC) 23(ALT ELPAC)
#testperiod blank
#testscoretype blank
#testname ListeningPL	SpeakingPL	ReadingPL	WritingPL 
#scalescore and plscore melt down
#proficiencylevelcode is hard mapping

#How often do you need this done?
#People dont start asking until end of the testing window, but want a monthly pull. As we lose enrollments of students, we lose access to their scores
#End of the month from Feb through May. Last day of Feb.

#Prioritize ELPAC pulldowns are Feb-May. 
#SBAC is April - June (mid) (End of April for SBAC/CAST)

#CAST has recordtypes that are taken apart within the SBAC file. 

#What table do they send it to?
# ELPAC [TestScores].[dbo].[vw_rpt_ELPACScores]
# CAST  [TestScores].[dbo].[CASTScores]
# SBAC: ELA/Math [TestScores].[dbo].[vw_rpt_SBACScores]